### Import Dependencies

In [1]:

import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from ConvLSTM3D_model.Seq2Seq import Seq2Seq
from torch.utils.data import DataLoader
import torch.nn.functional as f

import io
import imageio
from ipywidgets import widgets, HBox

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
class Normalize:
    def __init__(self, channels):
        self.channels=channels
        
        self.minimums=[]
        self.maximums=[]

    def normalize(self, data):
        self.minimums=[]
        self.maximums=[]
        data=np.array(data).swapaxes(0, 1)
        for i in range(0, self.channels):
            #print(np.min(channel), np.max(channel))
            min=np.min(data[i])
            max=np.max(data[i])

            if max-min==0:
                self.maximums.append(1)
                self.minimums.append(0)

                if max!=0:
                    data[i]=np.divide(data[i], max)


            else:


                
                data[i]=(data[i]-min)/(max-min)

                self.minimums.append(min)
                self.maximums.append(max)

            print(np.min(data[i]), np.max(data[i]))

        data=data.swapaxes(0,1)
        print(np.shape(data))
        return data, self.minimums, self.maximums
    
    def normalize_other(self, data):
        data=np.array(data).swapaxes(0, 1)
        for i in range(0, self.channels):
            data[i]=(data[i]-self.minimums[i])/(self.maximums[i]-self.minimums[i])

        data=data.swapaxes(0, 1)

        return data

        




In [5]:
data=np.float32(np.load("npys/data.npy")[:, :, :, 0:10, 100:132, 200:232])

np.random.shuffle(data)

train_data = data[:3]
val_data =  data[3:]

print(np.shape(train_data))
print(train_data.dtype)
#test_data = MovingMNIST[9000:10000]    
norm=Normalize(np.shape(train_data)[1])

normalized_train=norm.normalize(train_data)[0]
normalized_val=norm.normalize_other(val_data)



(3, 19, 3, 10, 32, 32)
float32
0.0 0.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 0.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 0.0
0.0 0.0
1.0 1.0
0.0 0.0
(3, 19, 3, 10, 32, 32)


In [6]:
print(np.shape(normalized_val))

(1, 19, 3, 10, 32, 32)


### Import Data and Create Dataloaders

In [7]:


#Dimensions are (Batch size, channels, frames, depth, length, width)
#shape is (B, 19, F, 47, 265, 442)


def collate(batch):

    
    batch = torch.tensor(batch) 
    
  
    batch = batch.to(device)                    

    

    

    
                  
    return batch[:,:,0:2], batch[:,:,2]     


# Training Data Loader
train_loader = DataLoader(normalized_train, shuffle=True, 
                        batch_size=3, collate_fn=collate)

# Validation Data Loader
val_loader = DataLoader(normalized_val, shuffle=True, 
                        batch_size=1, collate_fn=collate)



In [8]:
print(norm.minimums)
print(norm.maximums)

[0, 0.07280183, 9.76312e-05, 0.0138467075, 0.00018044972, 0, 0.0059213694, -12.071808, 1353.3984, -14.064547, -9.914224, -0.13143565, -99.8985, 36.494087, 0.0, 0, 0, 0, 0]
[1, 0.28189087, 0.05323999, 0.07388042, 0.02999676, 1, 0.016187405, 13.05246, 2000.9531, 12.00677, 17.70199, 0.34492007, -95.515045, 39.891464, 9.0, 1, 1, 1, 1]


In [9]:
print(np.min(normalized_train))
print(normalized_train.all()<=1)

0.0
True


### Instantiate Model, Optimizer and Loss

In [10]:
# The input video frames are grayscale, thus single channel
model = Seq2Seq(num_channels=np.shape(normalized_train)[1], num_kernels=64, 
kernel_size=(3, 3, 3), padding=(1, 1, 1), activation="relu", 
frame_size=(10, 32, 32), num_layers=2).to(device)

optim = Adam(model.parameters(), lr=1e-4)


criterion = nn.MSELoss()

### Train for about 20 epochs

In [11]:
num_epochs = 4

for epoch in range(1, num_epochs+1):
    
    train_loss = 0                                                 
    model.train()                                                  
    for batch_num, (input, target) in enumerate(train_loader, 1):  
        output = model(input)            
        loss = criterion(output.flatten(), target.flatten())       
        loss.backward()                                            
        optim.step()                                               
        optim.zero_grad()                                           
        train_loss += loss.item()                                 
    train_loss /= len(train_loader.dataset)                       

    val_loss = 0                                                 
    model.eval()                                                   
    with torch.no_grad():                                          
        for input, target in val_loader:                          
            output = model(input)                                   
            loss = criterion(output.flatten(), target.flatten())   
            val_loss += loss.item()                                
    val_loss /= len(val_loader.dataset)                            

    print("Epoch:{} Training Loss:{:.2f} Validation Loss:{:.2f}\n".format(
        epoch, train_loss, val_loss))

Epoch:1 Training Loss:0.05 Validation Loss:0.15

Epoch:2 Training Loss:0.05 Validation Loss:0.15

Epoch:3 Training Loss:0.04 Validation Loss:0.15

Epoch:4 Training Loss:0.03 Validation Loss:0.15

